In [1]:
   import tensorflow as tf
   print(f"TF version: {tf.__version__}")
   gpus = tf.config.list_physical_devices('GPU')
   if gpus:
       print("GPU is available")
       print("Num GPUs Available: ", len(gpus))
   else:
       print("No GPUs available in your system.")

2025-06-11 00:36:21.621042: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-11 00:36:21.675513: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749569781.706570   20772 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749569781.716500   20772 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749569781.764497   20772 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

TF version: 2.19.0
GPU is available
Num GPUs Available:  1


In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

df = pd.read_csv('~/Food_Delivery_Times.csv')

df = df.dropna()

X = df.drop(['Order_ID', 'Delivery_Time_min'], axis='columns')
y = df['Delivery_Time_min']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(f"훈련 데이터 수: {len(X_train)}개, 테스트 데이터 수: {len(X_test)}개")

categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('Traffic_Level', categorical_transformer, ['Traffic_Level']),
        ('Time_of_Day', categorical_transformer, ['Time_of_Day']),
        ('Vehicle_Type', categorical_transformer, ['Vehicle_Type']),
        ('Weather', categorical_transformer, ['Weather'])
    ],
    remainder='passthrough' 
)


model = Sequential()

model.add(Dense(8, activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])


pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', model)])

early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

history = pipeline.fit(X_train, y_train,
                       regressor__epochs=2000,
                       regressor__batch_size=1,
                       regressor__validation_split=0.1,
                       regressor__callbacks=[early_stopping],
                       regressor__verbose=1)


model.summary()

# Preprocess X_test before evaluating the model
X_test_processed = pipeline.named_steps['preprocessor'].transform(X_test)

loss, mae = model.evaluate(X_test_processed, y_test, verbose=0)
print(f"\n테스트 데이터에 대한 최종 Mean Absolute Error (MAE): {mae:,.2f}분")
print(f"-> 모델의 예측치가 실제값과 평균적으로 {mae:,.2f}분 정도 차이남을 의미합니다.")


predictions = pipeline.predict(X_test)

results_df = pd.DataFrame({
    '실제 시간': y_test.values.flatten(),
    '예측 시간': predictions.flatten()
})
results_df['차이'] = results_df['실제 시간'] - results_df['예측 시간']

print("\n--- 실제 시간과 예측 시간 비교 (상위 5개) ---")
print(results_df.head())


2025-06-11 02:13:02.245544: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-11 02:13:02.313258: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749575582.342012   28787 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749575582.351360   28787 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749575582.400972   28787 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

훈련 데이터 수: 706개, 테스트 데이터 수: 177개


I0000 00:00:1749575585.417584   28787 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9558 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070, pci bus id: 0000:01:00.0, compute capability: 8.9


Epoch 1/2000


I0000 00:00:1749575586.517607   28856 service.cc:152] XLA service 0x726ac80051c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1749575586.517743   28856 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4070, Compute Capability 8.9
2025-06-11 02:13:06.550820: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1749575586.642361   28856 cuda_dnn.cc:529] Loaded cuDNN version 90900


...

 Trainable params: 161 (644.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 324 (1.27 KB)


테스트 데이터에 대한 최종 Mean Absolute Error (MAE): 6.15분
-> 모델의 예측치가 실제값과 평균적으로 6.15분 정도 차이남을 의미합니다.
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step

--- 실제 시간과 예측 시간 비교 (상위 5개) ---
   실제 시간      예측 시간         차이
0     64  74.012711 -10.012711
1     54  51.170937   2.829063
2     52  58.638355  -6.638355
3     71  67.918045   3.081955
4     91  77.598724  13.401276


/home/minux/miniforge3/envs/jupyter_env/lib/python3.11/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
